In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# read data
top_dir = '/Users/stiso/Documents/Python/graphLearning/ECoG data/behavioral_data_raw/'
save_dir = '/Users/stiso/Documents/Python/graphLearning/ECoG data/behavior_preprocessed/'
subj = [1,2,4,6]

In [5]:
#%% Load and concatenate data

data = pd.DataFrame()
for i in subj:
    curr = pd.read_csv("".join([top_dir, 'subj', str(i), '/subj', str(i), '_clean_data']))
    curr['subj'] = i
    data = data.append(curr)
    

In [9]:
data.head(20)

,ISI_raw,Unnamed: 0,block,correct_raw,graph,hand,hand_transition,lag10,onset_raw,order,...,recency,resp,resp_raw,rt_raw,sess,subj,transition,trialNum,typing_raw,walk
0,1.0,1,1.0,1.0,1,1,1,1,1.0,1.0,...,1.0,1,1,1.0,1.0,1,1,1.0,1,1.0
1,1.0,1,1.0,1.0,1,1,1,1,1.0,1.0,...,1.0,1,1,1.0,1.0,1,1,1.0,1,1.0
2,1.0,1,1.0,1.0,1,1,1,1,1.0,1.0,...,1.0,1,1,1.0,1.0,1,1,1.0,1,1.0
3,1.0,1,1.0,1.0,1,1,1,1,1.0,1.0,...,1.0,1,1,1.0,1.0,1,1,1.0,1,1.0
4,1.0,1,1.0,1.0,1,1,1,1,1.0,1.0,...,1.0,1,1,1.0,1.0,1,1,1.0,1,1.0
5,1.0,1,1.0,1.0,1,1,1,1,1.0,1.0,...,1.0,1,1,1.0,1.0,1,1,1.0,1,1.0
6,1.0,1,1.0,1.0,1,1,1,1,1.0,1.0,...,1.0,1,1,1.0,1.0,1,1,1.0,1,1.0
7,1.0,1,1.0,1.0,1,1,1,1,1.0,1.0,...,1.0,1,1,1.0,1.0,1,1,1.0,1,1.0
8,1.0,1,1.0,1.0,1,1,1,1,1.0,1.0,...,1.0,1,1,1.0,1.0,1,1,1.0,1,1.0
9,1.0,1,1.0,1.0,1,1,1,1,1.0,1.0,...,1.0,1,1,1.0,1.0,1,1,1.0,1,1.0


In [8]:
# add session for subjects that dont have it
data[data['sess'].isna()] = 1

In [10]:
data.describe()

,ISI_raw,Unnamed: 0,block,correct_raw,lag10,onset_raw,order,pID,recency,rt_raw,sess,subj,trialNum,walk
count,4639.000000,4639.000000,4639.000000,4639.000000,4639.000000,4639.000000,4639.000000,4639.000000,4639.000000,4639.000000,4639.000000,4639.000000,4639.000000,4639.000000
mean,0.419023,305.570813,1.913343,0.994826,1.184523,275.458995,306.182367,3.658978,6.291227,0.822493,1.205648,3.658978,306.182367,3.001940
std,0.463077,330.580733,1.136175,0.071749,0.831678,305.047867,330.938859,2.242639,10.571382,0.239813,0.404218,2.242639,330.938859,2.792407
min,0.050000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.360932,1.000000,1.000000,1.000000,0.000000
25%,0.050000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.633818,1.000000,1.000000,1.000000,1.000000
50%,0.050000,184.000000,1.000000,1.000000,1.000000,152.534317,185.000000,4.000000,2.000000,0.829086,1.000000,4.000000,185.000000,1.000000
75%,1.000000,588.500000,3.000000,1.000000,1.000000,537.755737,589.500000,6.000000,6.000000,1.000000,1.000000,6.000000,589.500000,5.000000
max,1.000000,999.000000,4.000000,1.000000,5.000000,988.798218,1000.000000,6.000000,126.000000,5.751362,2.000000,6.000000,1000.000000,9.000000


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4639 entries, 0 to 1911
Data columns (total 22 columns):
ISI_raw            4639 non-null float64
Unnamed: 0         4639 non-null int64
block              4639 non-null float64
correct_raw        4639 non-null float64
graph              4639 non-null object
hand               4639 non-null object
hand_transition    4639 non-null object
lag10              4639 non-null int64
onset_raw          4639 non-null float64
order              4639 non-null float64
pID                4639 non-null float64
path               4639 non-null object
recency            4639 non-null float64
resp               4639 non-null object
resp_raw           4639 non-null object
rt_raw             4639 non-null float64
sess               4639 non-null float64
subj               4639 non-null int64
transition         4639 non-null object
trialNum           4639 non-null float64
typing_raw         4639 non-null object
walk               4639 non-null float64
dtype

In [12]:
data.to_csv("".join([save_dir, 'group_behavior.csv']))
